In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import glob

class People_Counter:
    def __init__(self, path):
        self.path = path
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.io.gfile.GFile(self.path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.compat.v1.Session(graph=self.detection_graph)

        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def detect(self, image):
        image_np_expanded = np.expand_dims(image, axis=0)
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})

        im_height, im_width, _ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0, i, 0] * im_height),
                             int(boxes[0, i, 1] * im_width),
                             int(boxes[0, i, 2] * im_height),
                             int(boxes[0, i, 3] * im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

if __name__ == "__main__":
    model_path = '/content/my_model.pb'
    peop_counter = People_Counter(path=model_path)
    threshold = 0.4
    no = 1
    for img_path in glob.glob("/content/images/test/*.jpg"):
        count = 0
        img = cv2.imread(img_path)
        img = cv2.resize(img, (640, 480))

        boxes, scores, classes, num = peop_counter.detect(img)

        for i in range(len(boxes)):
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                cv2.rectangle(img, (box[1], box[0]), (box[3], box[2]), (255, 0, 0), 2)
                count += 1
        cv2.putText(img, 'Count = ' + str(count), (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.imwrite("/content/test.jpg", img)
        no += 1
    print("\n\t\t\tSuccessfully saved all results!\n")



			Successfully saved all results!



In [ ]:
!wget -O model "https://drive.google.com/drive/folders/1IBgEyaASf10KUFTCbky9mtruUpyoqDWR?usp=drive_link"

--2024-04-22 05:40:27--  https://drive.google.com/drive/folders/1IBgEyaASf10KUFTCbky9mtruUpyoqDWR?usp=drive_link
Resolving drive.google.com (drive.google.com)... 172.217.214.113, 172.217.214.139, 172.217.214.101, ...
Connecting to drive.google.com (drive.google.com)|172.217.214.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘model’

model                   [  <=>               ] 246.65K   979KB/s    in 0.3s    

2024-04-22 05:40:28 (979 KB/s) - ‘model’ saved [252574]



In [ ]:
# Specify the path to the zip file
zip_file_path = '/content/my_model.pb.zip'

# Specify the directory where you want to extract the contents
extracted_dir_path = '/content'

# Unzip the file
import zipfile
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# Print a message indicating that the extraction is complete
print("Extraction completed.")


BadZipFile: File is not a zip file